In [1]:
import os 
import numpy as np
import local_utils
import hausdorff_from_dir 
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
2650*1000

2650000

In [2]:
#RP transforms 
def WS_sigma(sigma_f, por = 0.4, CEC=1,B0=4.5e-8, m=1.3):
    rho_grain = 2650*1000 #g/m^3
#     CEC = 1 #meq/g    1:smect,  .2:Ill,  .02-.09:Kaol
#     B0 = 4.78e-8  #m^2/(sV)
    F = por**(-m)
    Qv = rho_grain*((1-por)/por)*CEC
    B = B0*(1-.6*np.exp(-sigma_f/.013))
    sigma_b = 1/F*(sigma_f + B*Qv)
    return sigma_b,B,Qv


def HSU(conc_mat,CF_mat,mTDS=1.4200556641030946,bTDS=332.7093594248108,**kwargs):
    #kwargs fed to WS_sigma
    Cw = (mTDS*conc_mat*1000 + bTDS)/1e4
    sig_bs = WS_sigma(Cw,CEC=0,**kwargs)[0]
    sig_bc = WS_sigma(Cw,CEC=1,**kwargs)[0]
    return sig_bc*(1 - (3*(1-CF_mat)*(sig_bc-sig_bs))/(3*sig_bc - CF_mat*(sig_bc-sig_bs)))


def rock_physics(varlist,it,conc_mat):
    nlay,nrow,ncol = (26, 20, 100)
    dx,dy,dz = (30,30,3)
    Lx,Ly,Lz = (dx*ncol,dy*nrow,dz*nlay)

    CF_glob = varlist['CF_glob'][it]
    CF_var = varlist['CF_var'][it]
    corr_len = varlist['corr_len'][it]
    corr_len_yx = varlist['corr_len_yx'][it]
    corr_len_zx = varlist['corr_len_zx'][it]
    vario_type = varlist['vario_type'][it]
    lcol = int(corr_len/dx)
    llay = int(corr_len*corr_len_zx/dz)
    lrow = int(corr_len*corr_len_yx/dy)
    seed = varlist['seed'][it]
    por_mean = varlist['por_mean'][it]
    por_var  = varlist['por_var'][it]

    cf_mat = simulFFT(nrow,nlay, ncol,
                     CF_glob,
                     CF_var,
                     vario_type,
                     lrow , llay, lcol,seed=seed)
    cf_mat[cf_mat > 1.] = 1.
    cf_mat[cf_mat < 0.] = 0.

    por_mat = simulFFT(nrow,nlay, ncol,
                     por_mean,
                     por_var,
                     vario_type,
                     lrow , llay, lcol,seed=seed)
    por_mat[por_mat > 1.] = 1.
    por_mat[por_mat < 0.] = 0.
    sigma_bulk = HSU(conc_mat,cf_mat,por=por_mat)
    return sigma_bulk

In [13]:



sig_bulk = sig_bc*(1 - (3*(1-CF_mat)*(sig_bc-sig_bs))/(3*sig_bc - CF_mat*(sig_bc-sig_bs)))


sig_bulk =sig_bc - sig_bc(3*(1-CF_mat)*(sig_bc-sig_bs))/(3*sig_bc - CF_mat*(sig_bc-sig_bs))

rtho_bulk = 1/(sig_bc - sig_bc(3*(1-CF_mat)*(sig_bc-sig_bs))/(3*sig_bc - CF_mat*(sig_bc-sig_bs)))









0.8252640815001918

In [14]:
1/1e-3

1000.0

In [17]:
mTDS=1.4200556641030946
bTDS=332.7093594248108
conc_mat = 0
Cw = (mTDS*conc_mat*1000 + bTDS)/1e4

1/Cw

30.056262971645996

In [21]:
1/HSU(conc_mat,1,por=.3)

16.0193161670281